<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (RAY)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text from PDF and convert them into parquet files
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Doc_ID generation**: Each chunk is assigned a uniq id, based on content and hash
- **Exact Dedup**: Chunks with exact same content are filtered out
- **Fuzzy Dedup**: Eliminate chunks that are 'very similar' content
- **Doc quality**: Scores the documents based on criteria like number of words, if it contains bad words ..etc
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [1]:
import os
from my_config import MY_CONFIG

## RAY CONFIGURATION
num_cpus_available =  os.cpu_count()
# print (num_cpus_available)
# MY_CONFIG.RAY_NUM_CPUS = num_cpus_available // 2  ## use half the available cores for processing
MY_CONFIG.RAY_NUM_CPUS =  1
# print (MY_CONFIG.RAY_NUM_CPUS)
MY_CONFIG.RAY_MEMORY_GB = 2  # GB
# MY_CONFIG.RAY_RUNTIME_WORKERS = num_cpus_available // 3
MY_CONFIG.RAY_RUNTIME_WORKERS = 2

## Step-2:  Data

We will use white papers  about LLMs.  

- [Granite Code Models](https://arxiv.org/abs/2405.04324)
- [Attention is all you need](https://arxiv.org/abs/1706.03762)

You can of course substite your own data below

### 2.1 - Download data

In [2]:
import os, sys
import shutil
from utils import download_file

## Download the data files
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)

download_file (url = 'https://arxiv.org/pdf/1706.03762', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'attension.pdf' ))

download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite.pdf' ))


Local file 'input/attension.pdf' (2.22 MB) already exists. Skipping download.
Local file 'input/granite.pdf' (1.27 MB) already exists. Skipping download.


### 2.2  - Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_chunk_out')
output_docid_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_docid_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_exact_dedupe_out')
output_fuzzy_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '05_fuzzy_dedupe_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '06_embeddings_out')


## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


## Step-3: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 3.1 - Set Input/output Folder

In [4]:
STAGE = 1 

input_folder = MY_CONFIG.INPUT_DATA_DIR
output_folder =  output_parquet_dir

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'


### 3.2 -  Execute 

In [5]:
%%time 

import ast
import os
import sys

from data_processing_ray.runtime.ray import RayTransformLauncher
from data_processing.utils import GB, ParamsUtils

from pdf2parquet_transform import (
    pdf2parquet_contents_type_cli_param,
    pdf2parquet_contents_types,
)
from pdf2parquet_transform_python import Pdf2ParquetPythonTransformConfiguration
from pdf2parquet_transform_ray import Pdf2ParquetRayTransformConfiguration

# create parameters
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS, "memory": MY_CONFIG.RAY_MEMORY_GB * GB}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
ingest_config = {
    pdf2parquet_contents_type_cli_param: pdf2parquet_contents_types.JSON,
}

params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    "data_files_to_use": ast.literal_eval("['.pdf']"),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    "runtime_pipeline_id": "pipeline_id",
    "runtime_job_id": "job_id",
    "runtime_code_location": ParamsUtils.convert_to_ast(code_location),
}


sys.argv = ParamsUtils.dict_to_req(d=(params | ingest_config))
# create launcher
launcher = RayTransformLauncher(Pdf2ParquetRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")


00:25:24 INFO - pdf2parquet parameters are : {'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'double_precision': 8}
00:25:24 INFO - pipeline id pipeline_id
00:25:24 INFO - code location {'github': 'github', 'commit_hash': '12345', 'path': 'path'}
00:25:24 INFO - number of workers 2 worker options {'num_cpus': 1, 'memory': 2147483648, 'max_restarts': -1}
00:25:24 INFO - actor creation delay 0
00:25:24 INFO - job details {'job category': 'preprocessing', 'job name': 'pdf2parquet', 'job type': 'ray', 'job id': 'job_id'}
00:25:24 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
00:25:24 INFO - data factory data_ max_files -1, n_sample -1
00:25:24 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
00:25:24 INFO - Running locally
202

✅ Stage:1 completed successfully
CPU times: user 3.85 s, sys: 668 ms, total: 4.52 s
Wall time: 3min 13s


### 3.3 - Inspect Generated output

Here we should see one entry per input file processed

In [6]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

Output dimensions (rows x columns)=  (2, 12)


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,granite.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",28,17,348,81bc331a-69cf-49bd-84b9-afedcab1344a,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf
1,attension.pdf,"{""_name"":"""",""type"":""pdf-document"",""description...",15,4,193,7afd3fbc-3a9f-4728-8fd8-4a9a13980244,pdf,6fe23d4f932c725077dfc8334f3f4da4e3aaf908d2aa23...,135814,2024-10-02T00:26:29.888597,53.822026,attension.pdf


##  Step-4: Doc chunks

Split the documents in chunks, according to their layout segmentation.

### 4.1 - Set Input/output Folder

In [7]:
STAGE = 2

input_folder = output_parquet_dir # previous output folder is the input folder for the current stage
output_folder =  output_chunk_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_chunk_out'


### 4.2 -  Execute 

In [8]:
%%time 

# Import doc_json_chunk transform configuration
from doc_chunk_transform_ray import DocChunkRayTransformConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc_chunk arguments
    # ...
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(DocChunkRayTransformConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

00:28:36 INFO - doc_chunk parameters are : {'chunking_type': <chunking_types.DL_JSON: 'dl_json'>, 'content_column_name': 'contents', 'doc_id_column_name': 'document_id', 'dl_min_chunk_len': None, 'output_chunk_column_name': 'contents', 'output_source_doc_id_column_name': 'source_document_id', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox'}
00:28:36 INFO - pipeline id pipeline_id
00:28:36 INFO - code location None
00:28:36 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
00:28:36 INFO - actor creation delay 0
00:28:36 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_chunk', 'job type': 'ray', 'job id': 'job_id'}
00:28:36 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_chunk_out
00:28:36 INFO - data factory data_ max_files -1, n_sample -1
00:28:36 INFO - data factory data_ Not using data sets, 

✅ Stage:2 completed successfully
CPU times: user 992 ms, sys: 321 ms, total: 1.31 s
Wall time: 19.6 s


### 4.3 - Inspect Generated output

We would see documents are split into many chunks

In [9]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 2
Chunks created : 211
Input data dimensions (rows x columns)=  (2, 12)
Output data dimensions (rows x columns)=  (211, 16)


,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id
185,attension.pdf,15,4,193,pdf,6fe23d4f932c725077dfc8334f3f4da4e3aaf908d2aa23...,135814,2024-10-02T00:26:29.888597,53.822026,attension.pdf,7afd3fbc-3a9f-4728-8fd8-4a9a13980244,6.1 Machine Translation\nOn the WMT 2014 Engli...,$.main-text[108],8,"[107.27262115, 260.13467407, 505.24533081, 302...",d6c1d3686219a176bc5ff0ebf4f5c82a53d95d1502d476...
94,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,6.3 Code Editing and Translation\nFrom Table 1...,$.main-text[199],17,"[107.33219147, 356.5696106, 505.74539185, 411....",1c841522286ea1348acafd3a4cfbbffd327ca5de53c5f9...
175,attension.pdf,15,4,193,pdf,6fe23d4f932c725077dfc8334f3f4da4e3aaf908d2aa23...,135814,2024-10-02T00:26:29.888597,53.822026,attension.pdf,7afd3fbc-3a9f-4728-8fd8-4a9a13980244,5.1 Training Data and Batching\nWe trained on ...,$.main-text[91],7,"[107.12083435, 343.05245972, 505.65435791, 418...",77de84b7743b8360a371146c12c9795a12984ef82354f4...


## Step-5:  DOC ID generation

This transform annotates documents with document "ids". It supports the following transformations of the original data:

 - Adding document hash: this enables the addition of a document hash-based id to the data. The hash is calculated with `hashlib.sha256(doc.encode("utf-8")).hexdigest()`. To enable this annotation, set hash_column to the name of the column, where you want to store it.
 - Adding integer document id: this allows the addition of an integer document id to the data that is unique across all rows in all tables provided to the transform() method. To enable this annotation, set int_id_column to the name of the column, where you want to store it. **This is a pre-requisite for fuzzy dedup** in the pipeline.

### 5.1 - Set Input/output Folder

In [10]:

STAGE  = 3

input_folder = output_chunk_dir # previous output folder is the input folder for the current stage
output_folder =  output_docid_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-3: Processing input='output/02_chunk_out' --> output='output/03_docid_out'


### 5.2 - Execute 

In [11]:
%%time 

from doc_id_transform_ray import DocIDRayTransformRuntimeConfiguration
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # doc id configuration
    "doc_id_doc_column": "contents",
    "doc_id_hash_column": "chunk_hash",
    "doc_id_int_column": "chunk_id",
}
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(DocIDRayTransformRuntimeConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

00:28:55 INFO - Doc id parameters are : {'doc_column': 'contents', 'hash_column': 'chunk_hash', 'int_column': 'chunk_id', 'start_id': 0}
00:28:55 INFO - pipeline id pipeline_id
00:28:55 INFO - code location None
00:28:55 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
00:28:55 INFO - actor creation delay 0
00:28:55 INFO - job details {'job category': 'preprocessing', 'job name': 'doc_id', 'job type': 'ray', 'job id': 'job_id'}
00:28:55 INFO - data factory data_ is using local data access: input_folder - output/02_chunk_out output_folder - output/03_docid_out
00:28:55 INFO - data factory data_ max_files -1, n_sample -1
00:28:55 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
00:28:55 INFO - Running locally
2024-10-02 00:28:56,881	INFO worker.py:1744 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
(orchestrate pid=641

✅ Stage:3 completed successfully
CPU times: user 123 ms, sys: 167 ms, total: 290 ms
Wall time: 15 s


### 5.3 - Inspect Generated output

In [12]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (211, 16)
Output data dimensions (rows x columns)=  (211, 18)


,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_hash,chunk_id
31,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,3 Model Architecture\nremove final 8 layers fr...,$.main-text[69],6,"[107.45430756, 456.21582031, 504.50476074, 521...",72fbd93a7a834627114fd13cdb1a48c354d6bd991a9eb9...,72fbd93a7a834627114fd13cdb1a48c354d6bd991a9eb9...,119
116,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,Acknowledgments\nThanks and acknowledgement to...,$.main-text[249],21,"[107.07092285, 59.12960052, 505.24591064, 160....",b6d51d1a54147d95051f77bf536ca6ab7360102dd5ac84...,b6d51d1a54147d95051f77bf536ca6ab7360102dd5ac84...,204
95,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,6.3 Code Editing and Translation\nCodeLingua (...,$.main-text[200],17,"[107.03813934, 207.6650238, 505.74505615, 350....",c52299a48da2f5517c7ed6b964195a46dd0e339af1d0f3...,c52299a48da2f5517c7ed6b964195a46dd0e339af1d0f3...,183


## Step-6: Exact Dedup

Remove documents having identical code to remove bias in the training data. On the content of each document, a SHA256 hash is computed,
followed by de-duplication of record having identical hashes.

### 6.1 -  Set Input/output Folder

In [13]:
STAGE  = 4

input_folder = output_docid_dir # previous output folder is the input folder for the current stage
output_folder =  output_exact_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-4: Processing input='output/03_docid_out' --> output='output/04_exact_dedupe_out'


### 6.2 - Execute 

In [14]:
%%time

# Import ededup transform configuration
from ededup_transform_ray import EdedupRayTransformRuntimeConfiguration


# Prepare the commandline params
local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # ededup parameters
    "ededup_hash_cpu": 0.5,
    "ededup_num_hashes": 2,
    "ededup_doc_column": "contents",
    "ededup_doc_id_column": "chunk_hash",
    
}

# Pass the commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# create launcher
launcher = RayTransformLauncher(EdedupRayTransformRuntimeConfiguration())
# launch
return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

00:29:10 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'chunk_hash', 'use_snapshot': False, 'snapshot_directory': None, 'hash_cpu': 0.5, 'num_hashes': 2}
00:29:10 INFO - pipeline id pipeline_id
00:29:10 INFO - code location None
00:29:10 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
00:29:10 INFO - actor creation delay 0
00:29:10 INFO - job details {'job category': 'preprocessing', 'job name': 'ededup', 'job type': 'ray', 'job id': 'job_id'}
00:29:10 INFO - data factory data_ is using local data access: input_folder - output/03_docid_out output_folder - output/04_exact_dedupe_out
00:29:10 INFO - data factory data_ max_files -1, n_sample -1
00:29:10 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
00:29:10 INFO - Running locally
2024-10-02 00:29:11,920	INFO worker.py:1744 -- Started a local Ray instance. View the dash

✅ Stage:4 completed successfully
CPU times: user 120 ms, sys: 172 ms, total: 292 ms
Wall time: 14.9 s


### 6.3 - Inspect Generated output

In [15]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input chunks before exact dedupe : {input_df.shape[0]:,}")
print (f"Output chunks after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate chunks removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (211, 18)
Output data dimensions (rows x columns)=  (211, 19)
Input chunks before exact dedupe : 211
Output chunks after exact dedupe : 211
Duplicate chunks removed :   0


,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_hash,chunk_id,removed
188,attension.pdf,15,4,193,pdf,6fe23d4f932c725077dfc8334f3f4da4e3aaf908d2aa23...,135814,2024-10-02T00:26:29.888597,53.822026,attension.pdf,7afd3fbc-3a9f-4728-8fd8-4a9a13980244,6.2 Model Variations\nTo evaluate the importan...,$.main-text[112],8,"[107.1419754, 91.9256134, 504.05615234, 113.59...",6eb55d1014abb7e7a010fd07b994af17a0cad7ca059f8f...,6eb55d1014abb7e7a010fd07b994af17a0cad7ca059f8f...,65,[]
153,attension.pdf,15,4,193,pdf,6fe23d4f932c725077dfc8334f3f4da4e3aaf908d2aa23...,135814,2024-10-02T00:26:29.888597,53.822026,attension.pdf,7afd3fbc-3a9f-4728-8fd8-4a9a13980244,3.2.2 Multi-Head Attention\noutput values. The...,$.main-text[54],5,"[107.36427307, 696.97607422, 503.99719238, 717...",07f191b8e14ee3784ecc42c94e4096c97388733f1ea59b...,07f191b8e14ee3784ecc42c94e4096c97388733f1ea59b...,30,[]
68,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,"6.1.5 RepoBench, CrossCodeEval: Repository-Lev...",$.main-text[154],12,"[107.21151733, 141.59487915, 505.73928833, 218...",650d9bcdcb744b665a189a4d02f09a4be39dcde46a0ecd...,650d9bcdcb744b665a189a4d02f09a4be39dcde46a0ecd...,156,[]


## Step-7: Fuzzy Dedup

Post exact deduplication, fuzzy deduplication is applied with
the goal of removing code files that may have slight variations and thereby unbiasing
the data further. Small variations are quite commonly seen in code data in the form
of variations in the values of variables, addittion of logging statements etc. Find near-
duplicate.

### 7.1 - Set Input/output Folder

In [16]:
## Input to this component is the output of doc_id generator component. 

STAGE  = 5

input_folder = output_exact_dedupe_dir # previous output folder is the input folder for the current stage
output_folder =  output_fuzzy_dedupe_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-5: Processing input='output/04_exact_dedupe_out' --> output='output/05_fuzzy_dedupe_out'


### 7.2 - Execute 

In [17]:
%%time 

import os
import sys

from data_processing.utils import ParamsUtils
from fdedup_transform_ray import FdedupRayTransformConfiguration

# create parameters

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
code_location = {"github": "github", "commit_hash": "12345", "path": "path"}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # Orchestration parameters
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # columns used
    "fdedup_doc_column": "contents",
    "fdedup_id_column": "chunk_id",
    "fdedup_cluster_column": "chunk_hash",
    # infrastructure
    "fdedup_bucket_cpu": 0.3,
    "fdedup_doc_cpu": 0.3,
    "fdedup_mhash_cpu": 0.3,
    "fdedup_num_doc_actors": 1,
    "fdedup_num_bucket_actors": 1,
    "fdedup_num_minhash_actors": 1,
    "fdedup_num_preprocessors": 1,
    # fuzzy parameters
    "fdedup_num_permutations": 64,
    "fdedup_threshold": 0.7, # between 0.0 to 1.0 ; smaller values tend to be more lenient in finding near dupes; close to 1.0 is more strict
    "fdedup_shingles_size": 5,
    "fdedup_delimiters": " "
}

# Pass commandline params
sys.argv = ParamsUtils.dict_to_req(d=params)

# launch

launcher = RayTransformLauncher(FdedupRayTransformConfiguration())

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

00:29:25 INFO - fuzzy dedup params are {'doc_column': 'contents', 'id_column': 'chunk_id', 'cluster_column': 'chunk_hash', 'bucket_cpu': 0.3, 'mhash_cpu': 0.3, 'doc_cpu': 0.3, 'num_doc_actors': 1, 'num_minhash_actors': 1, 'num_bucket_actors': 1, 'num_preprocessors': 1, 'num_permutations': 64, 'threshold': 0.7, 'shingles_size': 5, 'delimiters': ' ', 'snapshot_delay': 1, 'use_bucket_snapshot': False, 'use_doc_snapshot': False, 'random_delay_limit': 10, 'worker_options': {'num_cpus': 1}}
00:29:25 INFO - pipeline id pipeline_id
00:29:25 INFO - code location None
00:29:25 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
00:29:25 INFO - actor creation delay 0
00:29:25 INFO - job details {'job category': 'preprocessing', 'job name': 'fdedup', 'job type': 'ray', 'job id': 'job_id'}
00:29:25 INFO - data factory data_ is using local data access: input_folder - output/04_exact_dedupe_out output_folder - output/05_fuzzy_dedupe_out
00:29:25 INFO - data factory data_ max

✅ Stage:5 completed successfully
CPU times: user 223 ms, sys: 189 ms, total: 412 ms
Wall time: 39 s


### 7.3 - Inspect Generated output

In [18]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)
print ("Duplicate chunks removed  by fuzzy-dedupe:  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (211, 19)
Output data dimensions (rows x columns)=  (211, 19)
Duplicate chunks removed  by fuzzy-dedupe:   0


,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_id,removed,chunk_hash
47,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,6.1.1 HumanEvalSynthesize: Multilingual Code G...,$.main-text[118],9,"[107.09940338, 505.84005737, 505.70474243, 604...",22dd65548755f19ec6ccd89020fd1fbc88e339fafbd881...,135,[],-1
134,attension.pdf,15,4,193,pdf,6fe23d4f932c725077dfc8334f3f4da4e3aaf908d2aa23...,135814,2024-10-02T00:26:29.888597,53.822026,attension.pdf,7afd3fbc-3a9f-4728-8fd8-4a9a13980244,1 Introduction\nAttention mechanisms have beco...,$.main-text[20],2,"[107.17721558, 497.6980896, 505.65536499, 540....",362722af4a10ed54ca21fd329149c01397a621e15f8306...,11,[],-1
93,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,6.3 Code Editing and Translation\nTarget Langu...,$.tables[13],17,"[161.45388794, 433.6942749, 450.61630249, 552....",f665c10385f0eb31b2b94e5e61c934651f5789f5ab528c...,181,[],-1


## Step-8:   Text encoding

Encode text for the vector storage.

### 8.1 - Set Input/output Folder

In [19]:
STAGE  = 6

input_folder = output_fuzzy_dedupe_dir
output_folder =  output_embeddings_dir

input_df = read_parquet_files_as_df(input_folder)  ## for debug purposes

print (f"🏃🏼 STAGE-{STAGE}: Processing input='{input_folder}' --> output='{output_folder}'")

🏃🏼 STAGE-6: Processing input='output/05_fuzzy_dedupe_out' --> output='output/06_embeddings_out'


### 8.2 - Execute

In [20]:
%%time 

from text_encoder_transform_ray import TextEncoderRayTransformConfiguration

local_conf = {
    "input_folder": input_folder,
    "output_folder": output_folder,
}
worker_options = {"num_cpus" : MY_CONFIG.RAY_NUM_CPUS}
params = {
    # where to run
    "run_locally": True,
    # Data access. Only required parameters are specified
    "data_local_config": ParamsUtils.convert_to_ast(local_conf),
    # orchestrator
    "runtime_worker_options": ParamsUtils.convert_to_ast(worker_options),
    "runtime_num_workers": MY_CONFIG.RAY_RUNTIME_WORKERS,
    # text_encoder
    "text_encoder_model_name": MY_CONFIG.EMBEDDING_MODEL,
}

sys.argv = ParamsUtils.dict_to_req(d=params)
# create launcher
launcher = RayTransformLauncher(TextEncoderRayTransformConfiguration())
# Launch the ray actor(s) to process the input

return_code = launcher.launch()

if return_code == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception ("❌ Ray job failed")

00:30:04 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
00:30:04 INFO - pipeline id pipeline_id
00:30:04 INFO - code location None
00:30:04 INFO - number of workers 2 worker options {'num_cpus': 1, 'max_restarts': -1}
00:30:04 INFO - actor creation delay 0
00:30:04 INFO - job details {'job category': 'preprocessing', 'job name': 'text_encoder', 'job type': 'ray', 'job id': 'job_id'}
00:30:04 INFO - data factory data_ is using local data access: input_folder - output/05_fuzzy_dedupe_out output_folder - output/06_embeddings_out
00:30:04 INFO - data factory data_ max_files -1, n_sample -1
00:30:04 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
00:30:04 INFO - Running locally
2024-10-02 00:30:06,760	INFO worker.py:1744 -- Started a local Ray instance.

✅ Stage:6 completed successfully
CPU times: user 638 ms, sys: 269 ms, total: 907 ms
Wall time: 29 s


### 8.3 - Inspect Generated output

In [21]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_folder)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (211, 19)
Output data dimensions (rows x columns)=  (211, 20)


,filename,num_pages,num_tables,num_doc_elements,ext,hash,size,date_acquired,pdf_convert_time,source_filename,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,chunk_id,removed,chunk_hash,embeddings
171,attension.pdf,15,4,193,pdf,6fe23d4f932c725077dfc8334f3f4da4e3aaf908d2aa23...,135814,2024-10-02T00:26:29.888597,53.822026,attension.pdf,7afd3fbc-3a9f-4728-8fd8-4a9a13980244,4 Why Self-Attention\nlength n is smaller than...,$.main-text[85],7,"[107.26034546, 652.83349609, 504.29177856, 717...",6f8efa86e0a4f77b0d72d4a3141e5e0611b2921a392b99...,48,[],-1,"[0.018015103, -0.038851, 0.0016827772, -0.0493..."
25,granite.pdf,28,17,348,pdf,79c53d694df467391e94f279af2fa6a9a7e45c3922546e...,655054,2024-10-02T00:28:23.836369,167.768806,granite.pdf,81bc331a-69cf-49bd-84b9-afedcab1344a,"3 Model Architecture\nBatch size, 3B = 2048. B...",$.tables[0],5,"[138.25450134, 299.99499512, 471.55078125, 432...",b8f3a83c697e885ad31913c716644399a4772691e39d0b...,113,[],-1,"[0.003977602, -0.06122852, -0.089708336, -0.00..."
137,attension.pdf,15,4,193,pdf,6fe23d4f932c725077dfc8334f3f4da4e3aaf908d2aa23...,135814,2024-10-02T00:26:29.888597,53.822026,attension.pdf,7afd3fbc-3a9f-4728-8fd8-4a9a13980244,"2 Background\nSelf-attention, sometimes called...",$.main-text[24],2,"[107.29702759, 256.18237305, 505.24960327, 298...",9c2abd2ec38b67c74873e0cd670d27b702711d05930f26...,14,[],-1,"[0.03394238, -0.0117239505, -0.03349689, -0.02..."


## Step-9: Copy output to final output dir

In [22]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_folder, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_folder}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/06_embeddings_out' --> 'output/output_final'
